In [ ]:
import os

# install latest pytorch prebuilt with the default prebuilt CUDA version (usually the latest)
!pip install torch==1.5.0 torchvision==0.6.0 cudatoolkit=10.1 -c torch

# install the latest mmcv-full
!pip install mmcv-full==1.3.4

# install mmdetection
!pip install mmdet==2.11.0

# install mmocr
#!git clone https://github.com/open-mmlab/mmocr.git
os.chdir("./mmocr-main")
#os.chdir("/content/drive/MyDrive/AICUP_SceneTextDetection/mmocr-main")

!pip install -r requirements.txt
!pip install -v -e .  # or "python setup.py develop"
!export PYTHONPATH=$(pwd):$PYTHONPATH
    
os.chdir("..")

In [ ]:
!python mmocr-main/tools/train.py mmocr-main/configs/textdet/maskrcnn/public_deform_mask_rcnn_101.py --work-dir mmocr-main/work_dirs/exp --no-validate

In [ ]:
!python mmocr-main/tools/test.py mmocr-main/configs/textdet/maskrcnn/private_deform_mask_rcnn_101.py mmocr-main/work_dirs/exp/best.pth --out mmocr-main/work_dirs/exp/private_ocr_results.pkl

In [ ]:
!python mmocr-main/tools/test.py mmocr-main/configs/textdet/maskrcnn/public_deform_mask_rcnn_101.py mmocr-main/work_dirs/exp/best.pth --out mmocr-main/work_dirs/exp/public_ocr_results.pkl

In [ ]:
import numpy as np 
import scipy as sp
import scipy.misc, scipy.ndimage.interpolation
import pandas as pd
import os
import nibabel as nib
import cv2
import matplotlib.pyplot as plt 
from skimage import data, exposure
from glob import glob
import json
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')

In [ ]:
import mmcv
import json
with open('PublicTestDataset/instances_public_test.json', 'r') as f:
    json_file = json.load(f)
results = mmcv.load('mmocr-main/work_dirs/exp/public_ocr_results.pkl', 'pkl')

preds = []
for idx, result in enumerate(results):
  for boundary in result['boundary_result']:
    if boundary[-1] > 0.6:
      pred = [idx + 1]
      pred.extend(np.array(boundary)[:-1].astype('int32').flatten())
      pred.append(boundary[-1])
      preds.append(pred)

with open('PrivateTestDataset/instances_private_test.json', 'r') as f:
    json_file = json.load(f)
results = mmcv.load('mmocr-main/work_dirs/exp/private_ocr_results.pkl', 'pkl')

for idx, result in enumerate(results):
  for boundary in result['boundary_result']:
    if boundary[-1] > 0.6:
      pred = [idx + 3001]
      pred.extend(np.array(boundary)[:-1].astype('int32').flatten())
      pred.append(boundary[-1])
      preds.append(pred)


pd.DataFrame(preds).to_csv('submission.csv', index=False, header=False)